## Trabajo RL Víctor Rodríguez Part
Para este trabajo intentaré entrenar a un agente que aprenda a invertir en bolsa en un '3rd party environment' de gymnassium. Específicamente gym-trading_env.

Para esto usaremos la técnica A2C. Específicamente importaré la clase A2C de la librería stable-baselines3.



In [263]:
import sys

sys.version

'3.10.13 (main, Sep 11 2023, 13:21:10) [GCC 11.2.0]'

1. Realizamos las importaciones.

In [264]:
import gymnasium as gym
import gym_trading_env
from gym_trading_env.downloader import download

from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import os
from collections import deque
from sklearn.linear_model import LinearRegression

import tensorflow as tf
import keras
from keras import layers, optimizers, regularizers



### 2. Instanciamos el environment. Pdemos encontrar la información en: https://gym-trading-env.readthedocs.io/en/latest/#

- Descargamos y guardamos los datos del mercado en la variable df con las recomendaciones de la documentación.
- Indicamos en position las acciónes (y porcentaje de la cartera) que puede invertir en cada acción. En este caso, lo dejaremos apalancarse ya que done = True cuando la cartera vale 0. Por lo que si no se apalanca, nunca podrá acabar el episodio. Entonces pondremos el rango de acciones entre -2 (short) y 2 (long). Tampoco queremos que el actor lo apueste todo cada vez, por lo que permitiremos al bot invertir en múltiplos del 25% de la cartera.
- Iniciamos la posición en 0 (todo USD)
- El 'observation space' consiste en:
    - Observaciones estáticas (el actor no las puede cambiar): Los datos del mercado introducidos en el data frame.
    - Observaciones dinámicas (el actor las puede cambiar): Última acción del agente y la posición actual.
- Según la documentación, tenemos que añadir al data frame, columnas con el nombre feature con los inputs que queramos pasarle a la red.

In [265]:

# # Download BTC/USDT historical data from Binance and stores it to directory ./data/binance-BTCUSDT-1h.pkl
# download(exchange_names = ["binance"],
#     symbols= ["BTC/USDT"],
#     timeframe= "1h",
#     dir = "data",
#     since= datetime(year= 2018, month= 1, day=1),
#     until = datetime(year= 2024, month= 6, day=29, hour=11),
# )
# # Import your fresh data


In [266]:
# df = pd.read_pickle("./data/binance-BTCUSDT-1h.pkl")

Añadimos las features recomendadas en la documentación.

In [267]:
# df["feature_close"] = df["close"].pct_change()
# df["feature_open"] = df["open"]/df["close"]
# df["feature_high"] = df["high"]/df["close"]
# df["feature_low"] = df["low"]/df["close"]
# df["feature_volume"] = df["volume"] / df["volume"].rolling(7*24).max()



Alternativamente, añadiré unas features que me parecen convenientes para representar la tendéncia de la última semana. Estas serán.

- Pendiente de la regresión lineal de la semana. Nos indica si estamos en una semana alcista o bajista.
- Máximo y mínimo de la semana. Veremos la amplitud de los precios de la semana.
- Precio de apertura de 7 días antes y precio de cierre de esta semana. Esto nos permitirá junto al resto de features aproximar la forma de la curva y ver si la tendencia es ascendente, descendente, cóncava, convexa...
- Distancia al máximo y al mínimo de la semana, para ver formas no lineales.

In [268]:
def calcular_pendiente(datos):
    x = np.arange(len(datos)).reshape(-1, 1)
    y = datos
    modelo = LinearRegression().fit(x, y)
    pendiente = modelo.coef_[0]
    return pendiente

In [269]:
# rolling_windows = df.rolling(window=7*24)
# df['feature_pendiente'] = rolling_windows['close'].apply(calcular_pendiente, raw=False)

In [270]:
# df['feature_price'] = df['close']
# df['feature_initial_price'] = df['open'].shift(7*24 - 1)
# df['feature_mid_price'] = df['open'].shift(7*12 - 1)
# df['feature_max_week'] = rolling_windows['high'].max()
# df['feature_min_week'] = rolling_windows["low"].min()
# # df['feature_distance_max'] = df['date_close'] - df[df['high']== df['feature_max_week']]['date_close'].iloc[0]
# # df['feature_distance_min'] = df[df["low"].rolling(7*24).max()]


# df.dropna(inplace= True)

He intentado sacar la diferencia de horas hasta la fecha pero no he podido, a cambio añado el precio a mitad semana, lo que ayuda a aproximar si es logarítmica, concava, convexa o exponencial

In [271]:
# # def find_max_index(x):
# #     if len(x) > 1:
# #         return x[:-1].idxmax()
# #     else:
# #         return x.idxmax()

# # # Encontrar el índice del máximo dentro de la ventana deslizante
# # rolling_windows = df.rolling(window=7*24, min_periods=1)
# # max_index = rolling_windows['high'].apply(find_max_index)
# # df['max_date_before'] = df.loc[max_index.values, 'date_close'].reset_index(drop=True)

In [272]:
# df

In [273]:
# #Crearemos la función actualizar_grafica para que, cada nueva hora,
# # la gráfica se vaya actualizando y el agente ya entrenado pueda invertir
# # o desinvertir.

# def actualizar_grafica(df):

#     try:
#         download(
#             exchange_names = ["binance"],
#             symbols= ["BTC/USDT"],
#             timeframe= "1h",
#             dir = "data",
#             since= (df['date_close'].iloc[-1] - timedelta(hours=2)),
#             until = (datetime.now() + timedelta(hours=1))
#         )

#     except:
#         return df
#     # new_rows = None
#     new_rows = pd.read_pickle("./data/binance-BTCUSDT-1h.pkl")

#     new_rows["feature_close"] = new_rows["close"].pct_change()
#     new_rows["feature_open"] = new_rows["open"]/new_rows["close"]
#     new_rows["feature_high"] = new_rows["high"]/new_rows["close"]
#     new_rows["feature_low"] = new_rows["low"]/new_rows["close"]

#     df = pd.concat([df, new_rows])
#     df['feature_volume'] = df['feature_volume'].fillna(df['volume'] / df['volume'].rolling(7*24, min_periods=1).max())
#     rolling_windows = df.rolling(window=7*24)
#     df['feature_pendiente'] = rolling_windows['close'].apply(calcular_pendiente, raw=False)
#     df['feature_price'] = df['close']
#     df['feature_initial_price'] = df['open'].shift(7*24 - 1)
#     df['feature_mid_price'] = df['open'].shift(7*12 - 1)
#     df['feature_max_week'] = rolling_windows['high'].max()
#     df['feature_min_week'] = rolling_windows["low"].min()
#     df.dropna(how = 'any', inplace= True)
#     return df

In [274]:
# df = actualizar_grafica(df)
# df

In [275]:
# df['feature_mid_price'] = df['open'].shift(7*12 - 1)
# df.to_csv('data/data_frame.csv')

Lo he comentado todo ya que tarda 2 minutos en ejecutar el código anterior y no necesito descargarme cada vez el código ya que me he descargado el df en formato csv

In [276]:
df = pd.read_csv('data/data_frame.csv', index_col='date_open')
df.index = pd.to_datetime(df.index)
df['date_close'] = pd.to_datetime(df['date_close'])
df

,open,high,low,close,volume,date_close,feature_close,feature_open,feature_high,feature_low,feature_volume,feature_pendiente,feature_price,feature_initial_price,feature_mid_price,feature_max_week,feature_min_week
date_open,,,,,,,,,,,,,,,,,
2018-01-14 23:00:00,13474.51,13600.80,13407.00,13474.99,569.639147,2018-01-15 00:00:00,0.001709,0.999964,1.009337,0.994954,0.093608,-10.103790,13474.99,16218.85,13385.10,16322.30,11400.0
2018-01-15 00:00:00,13477.98,13733.52,13460.30,13689.94,469.651827,2018-01-15 01:00:00,0.015952,0.984517,1.003183,0.983226,0.077177,-9.815875,13689.94,15941.93,13803.00,16100.00,11400.0
2018-01-15 01:00:00,13689.94,13712.75,13423.85,13460.94,620.132541,2018-01-15 02:00:00,-0.016728,1.017012,1.018707,0.997245,0.101906,-9.591784,13460.94,15850.08,13754.95,16100.00,11400.0
2018-01-15 02:00:00,13460.81,13519.89,13236.21,13307.48,612.851205,2018-01-15 03:00:00,-0.011400,1.011522,1.015962,0.994644,0.100709,-9.354020,13307.48,16030.01,14070.07,16031.99,11400.0
2018-01-15 03:00:00,13315.01,13448.72,13215.56,13368.43,422.526559,2018-01-15 04:00:00,0.004580,0.996004,1.006006,0.988565,0.069433,-9.174955,13368.43,15632.48,13950.05,15921.68,11400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-29 13:00:00,61045.43,61182.70,61010.00,61102.98,517.966610,2024-06-29 14:00:00,0.000943,0.999058,1.001305,0.998478,0.078869,-18.741506,61102.98,64287.96,62028.01,64521.00,58402.0
2024-06-29 14:00:00,61102.98,61160.00,60949.67,61018.01,542.430960,2024-06-29 15:00:00,-0.001391,1.001393,1.002327,0.998880,0.082594,-18.400452,61018.01,64307.09,61936.07,64521.00,58402.0
2024-06-29 15:00:00,61018.01,61106.00,60956.79,61106.00,364.187030,2024-06-29 16:00:00,0.001442,0.998560,1.000000,0.997558,0.055454,-18.029443,61106.00,64321.29,61712.96,64521.00,58402.0


In [277]:
#Definimos el espacio de acciones y creamos el entorno
positions = list(np.arange(-2, 2.01, 0.25))
positions[5] = 0 #La función crea un número muy cercano a 0, pero no es 0
env = gym.make("TradingEnv",
               df=df, positions = positions,
               initial_position= 0,
               borrow_interest_rate= 0.0003/100)

In [278]:
s0, info= env.reset()
s0, info

(array([ 1.7090396e-03,  9.9996436e-01,  1.0093366e+00,  9.9495435e-01,
         9.3608014e-02, -1.0103790e+01,  1.3474990e+04,  1.6218850e+04,
         1.3385100e+04,  1.6322300e+04,  1.1400000e+04,  0.0000000e+00,
         0.0000000e+00], dtype=float32),
 {'idx': 0,
  'step': 0,
  'date': numpy.datetime64('2018-01-14T23:00:00.000000000'),
  'position_index': 5,
  'position': 0,
  'real_position': 0,
  'data_close': 13474.99,
  'data_high': 13600.8,
  'data_date_close': Timestamp('2018-01-15 00:00:00'),
  'data_volume': 569.639147,
  'data_low': 13407.0,
  'data_open': 13474.51,
  'portfolio_valuation': 1000.0,
  'portfolio_distribution_asset': 0,
  'portfolio_distribution_fiat': 1000.0,
  'portfolio_distribution_borrowed_asset': 0,
  'portfolio_distribution_borrowed_fiat': 0,
  'portfolio_distribution_interest_asset': 0,
  'portfolio_distribution_interest_fiat': 0,
  'reward': 0})

### 3. Creamos un ejercicio aleatorio según las recomendaciones de la documentación.

In [279]:
done, truncated = False, False
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)

    #Esto es para renderizar los trades, pero es muy lento, por lo que de momento lo comento.
    # env.unwrapped.save_for_render(dir = "render_logs")

Market Return : 352.59%   |   Portfolio Return : -99.95%   |   


In [280]:
info

{'idx': 56465,
 'step': 56465,
 'date': numpy.datetime64('2024-06-29T17:00:00.000000000'),
 'position_index': 5,
 'position': 0,
 'real_position': 0.0,
 'data_close': 60986.84,
 'data_high': 61014.87,
 'data_date_close': Timestamp('2024-06-29 18:00:00'),
 'data_volume': 189.74272,
 'data_low': 60937.03,
 'data_open': 61009.91,
 'portfolio_valuation': 0.5399505704948573,
 'portfolio_distribution_asset': 0,
 'portfolio_distribution_fiat': 0.5399505704948573,
 'portfolio_distribution_borrowed_asset': 0,
 'portfolio_distribution_borrowed_fiat': 0,
 'portfolio_distribution_interest_asset': 0.0,
 'portfolio_distribution_interest_fiat': 0.0,
 'reward': -2.2204460492503136e-16}

In [281]:
#Lo mismo

# from gym_trading_env.renderer import Renderer
# renderer = Renderer(render_logs_dir="render_logs")
# renderer.run()

### 4. Definimos el modelo y los hiperparámetros.

In [282]:
def create_q_model():
    inputs = layers.Input(shape=s0.shape)

    layer1 = layers.Dense(32, activation='relu', activity_regularizer=regularizers.L2(0.01))(inputs)
    layer2 = layers.Dense(64, activation='relu', activity_regularizer=regularizers.L2(0.01))(layer1)
    layer3 = layers.Dense(128, activation='relu', activity_regularizer=regularizers.L2(0.01))(layer2)
    layer4 = layers.Dense(32, activation='relu', activity_regularizer=regularizers.L2(0.01))(layer3)


    action = layers.Dense(env.action_space.n, activation='linear', activity_regularizer=regularizers.L2(0.01))(layer4)

    return keras.Model(inputs=inputs, outputs=action)

In [283]:
decay = 0.99999
epsilon_ini = 1.0
epsilon_end = 0.05
learning_rate = 0.001
gamma = 0.999
max_memory_size = int(5e4)
batch_size = 16

In [284]:
from keras import backend as K
K.clear_session()

In [285]:
q_model = None
q_model = create_q_model()

optimizer = optimizers.Adam(learning_rate=learning_rate)
q_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 13)]              0         
                                                                 
 dense (Dense)               (None, 32)                448       
                                                                 
 dense_1 (Dense)             (None, 64)                2112      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 32)                4128      
                                                                 
 dense_4 (Dense)             (None, 17)                561       
                                                                 
Total params: 15,569
Trainable params: 15,569
Non-trainable p

In [286]:
def get_best_action(state):
    q_state = q_model(state)
    return np.argmax(q_state)

In [287]:
s0 = np.expand_dims(s0, axis=0)
q0 = q_model(s0)
q0

<tf.Tensor: shape=(1, 17), dtype=float32, numpy=
array([[  703.76434 ,    25.399372, -1239.7556  ,  -303.79214 ,
          239.75357 ,  -660.9809  ,  -894.75146 ,   605.11847 ,
          498.74518 ,  1895.8964  , -1349.3849  ,  -773.34106 ,
         -647.3785  ,   130.28795 ,   -57.748573,  -521.3033  ,
          563.36945 ]], dtype=float32)>

In [288]:
# implementation = 'DQN_TRADING_v0'
# current_time = datetime.now().strftime('%Y%m%d-%H%M%S')
# train_log_dir = os.path.join('logs', implementation, 'T_' + current_time)

In [289]:
# %load_ext tensorboard
# %tensorboard --logdir ./logs

In [290]:
memory = deque(maxlen=max_memory_size)

num_episodes = 15
epsilon = epsilon_ini
agent_eval = 0

done, truncated = False, False

for i in range(num_episodes):
    state, info= env.reset()
    state =np.expand_dims(state, axis=0)
    epsilon = epsilon ** 0.4 #Añado esta línea para que el epsilon aumente tras cada episodio pero no llegue a ser tan alto cómo el inicial.
    epoque = 0

    done, truncated = False, False
    while not done and not truncated:

        # if epoque == 1:
        #     print('Entra 1')

        if np.random.random() < epsilon:
            position_index = env.action_space.sample()
            # if epoque == 1:
            #     print('Entra 2')
        else:
            q0 = q_model(state)
            position_index = np.argmax(q0)

            # if epoque == 1:
            #     print('Entra 3')

        if epsilon > epsilon_end:
            epsilon *= decay

            # if epoque == 1:
            #     print('Entra 4')
        else:
            epsilon = epsilon_end
            # if epoque == 1:
            #     print('Entra 5')

        next_state, reward, done, truncated, info = env.step(position_index)
        next_state = np.expand_dims(next_state, axis=0)
        
        experience = (state, position_index, next_state, reward, done)
        memory.append(experience)

        # if epoque == 1:
        #     print('Entra 6')
        #     print(memory[-1], info)


        with tf.GradientTape() as tape:

            q_next = np.max(q_model(next_state))
            q_bellman = reward + gamma * q_next * (1-done)
            q_current = q_model(state)[0, position_index]
            loss = (q_current - q_bellman)**2
            grads = tape.gradient(loss, q_model.trainable_variables)
            optimizer.apply_gradients(zip(grads, q_model.trainable_variables))
            # if epoque == 1:
            #     print('Entra 7')
            #     print(f'Q_next: {q_next}, \n Q_bellman: {q_bellman}, , \n Q_current: {q_current},, \n loss:  {loss}, , \n grads: {grads}')
        epoque += 1
        if epoque % 1000 == 0:
            print(f'Epoca {epoque}: {memory[-1]}, {epsilon}')
            
        
        state = next_state
    print(f'Episode {i+1}: reward = {reward}, info: {info}')

Epoca 1000: (array([[-3.8076087e-03,  1.0038222e+00,  1.0063928e+00,  9.9478787e-01,
         1.5040162e-01, -1.1087266e+01,  1.0324000e+04,  1.1147110e+04,
         1.0180060e+04,  1.1786010e+04,  9.2747998e+03,  1.2500000e+00,
         1.2511965e+00]], dtype=float32), 6, array([[ 4.6619526e-03,  9.9507046e-01,  1.0045863e+00,  9.9257338e-01,
         1.3478339e-01, -1.0820497e+01,  1.0372130e+04,  1.1409000e+04,
         1.0204930e+04,  1.1786010e+04,  9.2747998e+03, -5.0000000e-01,
        -5.0350654e-01]], dtype=float32), -0.002334452486517505, False), 0.9900497842463938
Epoca 2000: (array([[ 1.1454005e-03,  9.9985456e-01,  1.0065322e+00,  9.9673986e-01,
         2.3057564e-01, -1.7453004e+00,  6.7389800e+03,  7.0207202e+03,
         6.7552402e+03,  7.5200000e+03,  6.5000000e+03,  1.5000000e+00,
         1.4991447e+00]], dtype=float32), 3, array([[ 2.3727627e-03,  9.9763286e-01,  1.0007298e+00,  9.9482453e-01,
         1.5402091e-01, -1.7728695e+00,  6.7549702e+03,  7.0126001e+03,


In [291]:
epsilon

0.39020344255531714

Probar un episodio solo explotando.

In [292]:
done, truncated = False, False
state, info= env.reset()
state =np.expand_dims(state, axis=0)
epoca = 1
while not done and not truncated:
    epoca += 1
    q0 = q_model(state)
    position_index = np.argmax(q0)
    observation, reward, done, truncated, info = env.step(position_index)

    if epoca % 500 == 0:
        print (info)

{'idx': 499, 'step': 499, 'date': numpy.datetime64('2018-02-04T18:00:00.000000000'), 'position_index': 0, 'position': -2.0, 'real_position': -0.7037654470870481, 'data_close': 8349.04, 'data_high': 8585.0, 'data_date_close': Timestamp('2018-02-04 19:00:00'), 'data_volume': 1179.225446, 'data_low': 8300.0, 'data_open': 8474.68, 'portfolio_valuation': 1760.8057524168848, 'portfolio_distribution_asset': 0, 'portfolio_distribution_fiat': 3000.0, 'portfolio_distribution_borrowed_asset': 0.14842311571288735, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 4.45269347138662e-07, 'portfolio_distribution_interest_fiat': 0.0, 'reward': 0.011266610304930041}
{'idx': 999, 'step': 999, 'date': numpy.datetime64('2018-02-26T23:00:00.000000000'), 'position_index': 0, 'position': -2.0, 'real_position': -1.0440490437399645, 'data_close': 10324.0, 'data_high': 10390.0, 'data_date_close': Timestamp('2018-02-27 00:00:00'), 'data_volume': 1001.443723, 'data_low': 10270.19,

In [293]:
state, info= env.reset()
info

{'idx': 0,
 'step': 0,
 'date': numpy.datetime64('2018-01-14T23:00:00.000000000'),
 'position_index': 5,
 'position': 0,
 'real_position': 0,
 'data_close': 13474.99,
 'data_high': 13600.8,
 'data_date_close': Timestamp('2018-01-15 00:00:00'),
 'data_volume': 569.639147,
 'data_low': 13407.0,
 'data_open': 13474.51,
 'portfolio_valuation': 1000.0,
 'portfolio_distribution_asset': 0,
 'portfolio_distribution_fiat': 1000.0,
 'portfolio_distribution_borrowed_asset': 0,
 'portfolio_distribution_borrowed_fiat': 0,
 'portfolio_distribution_interest_asset': 0,
 'portfolio_distribution_interest_fiat': 0,
 'reward': 0}

### Conclusiones:
- El modelo aprende a invertir siempre en contra del mercado. Esto se puede deber a que se empieza a entrenar el modelo con datos a partir de 2018.Otro motivo por el que puede estar entrenandose mal es porque las tendencias son más a largo plazo que a corto. Por tanto, un cambio de tendencia a largo plazo es difícil de predecir.
- Otro patrón que he observado es que , aunque al final de cada episodio, el modelo sigue entrenando con acerca del 60% de las experiéncias, este, entrena mucho más con datos viejos (de 2018) y explota más con datos de 2024, por lo que esto podríamos cambiarlo poniendo un decay bastante grande (para evitar tener que entrenar el modelo durante un tiempo excesivo) pero que solo cambie en al final del bucle indexado (en este código, el que empieza con while not done and not truncated...)
- También debido a la razón anterior, cómo el modelo tiene más dinero al principio que al final, esto puede afectar a sus inversiones y por tanto, entrenar mejor en las primeras experiéncias (que refuerza el comportamiento de tomar siempre la acción 0 (-2)). Esto se podría solucionar poniendo un  segundo bucle con un decay negativo, es decir, que explota mucho al principio (lo que haría que mantuviese el dinero) y que vaya aumentando poco a poco. Lo que podría hacer que llegase al centro del episodio con más dinero. (O incluso se podrían hacer tres bucles, uno con decay positivo, uno con un decay parabólico y uno negativo)

Mejoras para próximas iteraciones:
- Puesto que el DQN es un modelo basado en el bootstrapping, ente otras características, es un modelo inestable que no asegura la convergencia, en cambio, el A2C, que no requiere bootstrapping, converge en una buena solución, por lo que se podría probar a hacer modelos más refinados, cómo el doble DQN o diréctamente el cambio de paradigma al policy gradient cómo podría ser el A2C o A3C.


In [1]:
#He cerrado el vscode antes de guardar el modelo :(
q_model.save('pesos.keras', overwrite=True)

NameError: name 'q_model' is not defined